In [ ]:
import qsharp
import os, sys

notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)
from util import plot

In [ ]:
%%qsharp

operation Entangled(measurement_basis : Pauli) : Result[] {
    use (qubit1, qubit2) = (Qubit(), Qubit());

    // create entangled state
    H(qubit1);
    CNOT(qubit1, qubit2);

    // measure in any basis, as long as it is the same for both qubits
    let result1 = Measure([measurement_basis], [qubit1]);
    let result2 = Measure([measurement_basis], [qubit2]);
    ResetAll([qubit1, qubit2]);

    // the measurement results will be perfectly correlated
    [result1, result2]
}

![](entanglement2.excalidraw.svg)

In [ ]:
results = qsharp.run("Entangled(PauliZ)", shots=1000)
plot(results, title="Entangled electrons spin measurement - Z basis")

In [ ]:
results = qsharp.run("Entangled(PauliX)", shots=1000)
plot(results, title="Entangled electrons spin measurement - X basis", basis="X")

![](teleportation2.excalidraw.svg)

In [ ]:
%%qsharp

import Std.Convert.*;
import Std.Math.*;

operation TeleportationEndToEnd() : Result {
    // Step 1:
    // Allocate three qubits
    //  - alice_message: Alice’s unknown state
    //  - alice_qubit: Alice’s half of the entangled pair
    //  - bob_qubit: Bob’s half of the entangled pair
    use (alice_message, alice_qubit, bob_qubit) = (Qubit(), Qubit(), Qubit());

    // Step 2:
    // Alice and Bob entangle their qubits (alice_qubit and bob_qubit)
    H(alice_qubit);
    CNOT(alice_qubit, bob_qubit);

    // Step 3: 
    // Alice prepares some arbitrary state on 'message'
    PrepareState(alice_message);

    // Step 4:
    // Alice performs a Bell-basis measurement on (alice_message, alice_qubit)
    // and sends the results to Bob.
    let (m1, m2) = AliceTeleport(alice_message, alice_qubit);

    // Step 5:
    // Bob receives Alice's measurement results and applies X and/or Z depending on Alice’s bits m1, m2
    // to recover the original state on his qubit (bob_qubit).
    BobDecodeTeleportedState(m1, m2, bob_qubit);

    // Verification:
    // To check correctness - undo Alice’s preparation on Bob's 'target'
    Adjoint PrepareState(bob_qubit);
    let result = M(bob_qubit);

    ResetAll([alice_message, alice_qubit, bob_qubit]);

    result
}

// Alice performs a Bell-basis measurement on (message, resource), yielding two classical bits (m1, m2)
operation AliceTeleport(alice_message : Qubit, alice_qubit : Qubit) : (Bool, Bool) {
    CNOT(alice_message, alice_qubit);
    H(alice_message);

    let m1 = M(alice_message) == One;
    let m2 = M(alice_qubit) == One;

    (m1, m2)
}

/// Prepares an arbitrary single-qubit state (Alice's secret message).
operation PrepareState(q : Qubit) : Unit is Adj + Ctl {
    Rx(1. * PI() / 2., q);
    Ry(2. * PI() / 3., q);
    Rz(3. * PI() / 4., q);
}

/// Bob's decoding: apply X and/or Z depending on Alice's bits m1, m2
operation BobDecodeTeleportedState(m1 : Bool, m2 : Bool, bob_qubit : Qubit) : Unit {
    if not m1 and not m2 {
        // (0,0): do nothing
        I(bob_qubit);
    }
    if not m1 and m2 {
        // (0,1): apply X
        X(bob_qubit);
    }
    if m1 and not m2 {
        // (1,0): apply Z
        Z(bob_qubit);
    }
    if m1 and m2 {
        // (1,1): apply Z then X
        Z(bob_qubit);
        X(bob_qubit);
    }
}


In [ ]:
results = qsharp.run("TeleportationEndToEnd()", shots=1000)
success = results.count(qsharp.Result.Zero) / len(results)
print(f"Success rate of teleportation: {success:.2%}")